In [1]:

from my_modules.log import log,Limit,CounterBreak,CounterMod



from my_modules.load_data import MyDataset
import my_modules.model.lstm as model_modules
from my_modules.model.lstm import LSTM_0,move_to_gpu,del_gpu,LSTM_1,LSTM_2,LSTM_3,LSTM_4
from my_modules.model.lstmcnn_part2 import LSTMCNN_6,LSTMCNN_7,LSTMCNN_8,LSTMCNN_9


import torch
from torch import nn
from torch.utils.data import DataLoader

import os
# os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

from tqdm import tqdm

print(model_modules.get_used_memory())

0


In [2]:
train_index_movie = [i for i in range(38)]
test_index_movie = [i for i in range(len(train_index_movie),40)]

data_path = r'C:\Users\rakna\Desktop\AIT_working\AIT_CP_Project\this_folder_git_ignore\ori_datasets\s01.dat'
# window_size = 145*5
window_size = 145*10
step_window = 10

data_train = MyDataset(data_path,window_size,step_window,train_index_movie)
data_test = MyDataset(data_path,window_size,step_window,test_index_movie)

In [3]:
model_modules.do_gpu = torch.cuda.is_available()
# model_modules.do_gpu = False

if(model_modules.do_gpu):
    print(torch.cuda.get_device_name())
else:
    print('using CPU')

data_loader = DataLoader(data_train,batch_size = 16,shuffle=True,pin_memory=model_modules.do_gpu)
data_loader_test  = DataLoader(data_test,batch_size = 16,shuffle=True,pin_memory=model_modules.do_gpu)

GeForce GTX 1070 Ti


In [4]:
# input_size = num EEG channels


input_size = 40
torch.manual_seed(42)
save_folder = 'v015'
model = LSTMCNN_9(input_size=input_size,number_layers=1,hidden_size=20)
model = move_to_gpu(model)
# print(model.eval())
loss_func = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters_with_l2(),lr=1e-5,weight_decay=1e-3)
optimizer_without_l2 = torch.optim.Adam(model.parameters_without_l2(),lr=1e-5)


In [5]:
best_loss = None

pl_1 = Limit(1)
pl_1_2 = Limit(1)

do_train = True
is_limit = False
if(do_train):
    epoch = -1
    while True:
        epoch+=1
#     for epoch in range(100):

        pl = Limit(3)
        loss_train_batch = 0
        c=0
        for train_data,label in tqdm(data_loader):

            if(is_limit):
                if(CounterBreak.set_count(2).count()):
                    pl_1('limit train_data')
                    break
   
            train_data = move_to_gpu(train_data)
            label = move_to_gpu(label)


            optimizer.zero_grad()
            optimizer_without_l2.zero_grad()
            bz = train_data.shape[0]
            model.reset_hidden(bz)



            y_hat = (model(train_data))
          
            loss = loss_func(y_hat,label)
            loss_train_batch+=float(loss)
            c+=1
            loss.backward()
            optimizer.step()
            optimizer_without_l2.step()

    #         del_gpu(train_data,label)

        log(loss_train_batch/c,title=f"loss train at epoch {epoch}")
        
        

        '''
            find loss test
        '''
        loss_test=0
        c = 0
        with torch.no_grad():
            for test_data,y_test in tqdm(data_loader_test):
                
                
                if(is_limit):
                    if(CounterBreak.set_count(2).count()):
                        pl_1_2('limit test_data')
                        break

                test_data = move_to_gpu(test_data)    
                y_test = move_to_gpu(y_test)

                bz = test_data.shape[0]
                
                model.reset_hidden(bz)
                y_hat_test = model(test_data)

                batch_loss_test = loss_func(y_hat_test,y_test)

                loss_test+= float(batch_loss_test)
                c+=1

    #             del_gpu(test_data,y_test)
    
    
        loss_test = loss_test/c
        
        rp = r"C:\Users\rakna\Desktop\AIT_working\AIT_CP_Project\this_folder_git_ignore\w"
        if(CounterMod.init().set_mod(5).count()):
            log(loss_test,title=f"(from CounterMod)loss test at epoch {epoch}")
            log('ram gpu',torch.cuda.memory_allocated())
            torch.save(model.state_dict(), os.path.join(rp,save_folder,f'./autosave_model_{epoch:03}_{"{:.2f}".format(loss_test).replace(".","dot") }' ) )
            
        if(best_loss is None or loss_test<best_loss):
            best_loss=loss_test
            
            torch.save(model.state_dict(), os.path.join(rp,save_folder,f'./model_{epoch:03}_{"{:.2f}".format(loss_test).replace(".","dot") }' ) )
            log(loss_test,title=f"loss test at epoch {epoch}")
            log(title='save w',n=60)
        

        
        
    

    

  4%|███                                                                                | 3/83 [00:00<00:02, 29.70it/s]

==============================loss train at epoch 0==============================
21.840885894433452


  0%|                                                                                 | 2/1573 [00:00<01:48, 14.49it/s]

==============================loss test at epoch 0==============================
18.617691292820208
============================================================save w============================================================


  4%|███                                                                                | 3/83 [00:00<00:02, 28.85it/s]

...

==============================loss train at epoch 16==============================
7.164367731407199


  5%|████                                                                               | 4/83 [00:00<00:02, 31.50it/s]

==============================loss train at epoch 17==============================
6.955782974289241


 11%|████████▉                                                                      | 177/1573 [00:11<01:30, 15.46it/s]


KeyboardInterrupt: 